In [2]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from PIL import Image
import numpy as np
import csv
import pandas as pd
import keras.utils as image
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

2022-12-13 15:38:22.150538: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-13 15:38:23.233317: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-13 15:38:23.233617: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-13 15:38:23.233623: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [5]:
LETTERSTR = "0123456789abcdefghijklmnopqrstuvwxyz"


def toonehot(text):
    labellist = []
    for letter in text:
        onehot = np.array([0 for _ in range(36)])
        num = LETTERSTR.find(letter)
        onehot[num] = 1
        labellist.append(onehot)
    return np.array(labellist)


# Create CNN Model
print("Creating CNN model...")
in_ = Input((100, 100, 3))
out = in_
out = Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(out)
out = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)
out = Dropout(0.3)(out)
out = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(out)
out = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)
out = Dropout(0.3)(out)
out = Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu')(out)
out = Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)
out = Dropout(0.3)(out)
out = Conv2D(filters=256, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)
out = Flatten()(out)
out = Dropout(0.3)(out)
out = [Dense(36, name='digit1', activation='softmax')(out),\
    Dense(36, name='digit2', activation='softmax')(out)]
model = Model(inputs=in_, outputs=out)
model.compile(loss='categorical_crossentropy', optimizer='Adamax', metrics=['accuracy'])
model.summary()


train_data = []
train_label = [[] , []]
path = 'captcha-hacker/train/task2'
df = pd.read_csv('captcha-hacker/train/annotations.csv')
for filename in os.listdir(path):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        img = image.load_img(os.path.join(path , filename) , target_size = (100 , 100))
        aug_images = []
        img = np.expand_dims(img , 0)
        gen = ImageDataGenerator(
        rotation_range = 10 , 
        width_shift_range = 0.15 , 
        height_shift_range = 0.15 , 
        shear_range = 0.15 , 
        zoom_range = 0.1 , 
        channel_shift_range = 10 , 
        )
        aug_iter = gen.flow(img)
        aug_images = [next(aug_iter)[0].astype(np.uint8) for i in range(8)]
        label = df.loc[df['filename'] == os.path.join('task2' , filename) , 'label'].item()
        label_ = toonehot(label)
        for aug_pic in np.array(aug_images):
            aug_pic = aug_pic.astype('float')/255.0
            train_data.append(aug_pic)
            for i in range(2):
                train_label[i].append(label_[i])
train_data = np.array(train_data)
train_label = [arr for arr in np.asarray(train_label)]
# print(train_data.shape)
# print(train_label.shape)
# train_data = tf.stack(train_data)
# train_label = tf.stack(train_label)
# print (train_data.shape)

model.fit(train_data, train_label, batch_size = 100, epochs = 40, verbose=1 , validation_split=0.2) 
model.save("part2_model.h5")         

Creating CNN model...
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_14 (Conv2D)             (None, 100, 100, 32  896         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_15 (Conv2D)             (None, 98, 98, 32)   9248        ['conv2d_14[0][0]']              
                                                                      

2022-12-13 15:43:37.552432: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1920000000 exceeds 10% of free system memory.
2022-12-13 15:43:38.659064: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1920000000 exceeds 10% of free system memory.


Epoch 1/40
  2/160 [..............................] - ETA: 8s - loss: 12.4708 - digit1_loss: 6.3800 - digit2_loss: 6.0909 - digit1_accuracy: 0.0600 - digit2_accuracy: 0.0400      

2022-12-13 15:43:39.907207: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_2/dropout_8/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


160/160 [==============================] - 10s 59ms/step - loss: 8.2467 - digit1_loss: 4.1056 - digit2_loss: 4.1410 - digit1_accuracy: 0.0746 - digit2_accuracy: 0.0697 - val_loss: 8.1551 - val_digit1_loss: 4.1137 - val_digit2_loss: 4.0414 - val_digit1_accuracy: 0.0000e+00 - val_digit2_accuracy: 0.0192
Epoch 2/40
160/160 [==============================] - 9s 56ms/step - loss: 6.4164 - digit1_loss: 3.1669 - digit2_loss: 3.2495 - digit1_accuracy: 0.1748 - digit2_accuracy: 0.1568 - val_loss: 7.9611 - val_digit1_loss: 3.9901 - val_digit2_loss: 3.9710 - val_digit1_accuracy: 0.0465 - val_digit2_accuracy: 0.0490
Epoch 3/40
160/160 [==============================] - 9s 56ms/step - loss: 5.2195 - digit1_loss: 2.5511 - digit2_loss: 2.6684 - digit1_accuracy: 0.3038 - digit2_accuracy: 0.2801 - val_loss: 5.5803 - val_digit1_loss: 2.8081 - val_digit2_loss: 2.7723 - val_digit1_accuracy: 0.2148 - val_digit2_accuracy: 0.2145
Epoch 4/40
160/160 [==============================] - 9s 56ms/step - loss: 4.13